In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras.utils

  Preparing metadata (setup.py) ... done
  Created wheel for keras.utils: filename=keras_utils-1.0.13-py3-none-any.whl size=2631 sha256=a4cd759e375e8b38127702bb8faa2e17e91d0444202b17fbda7e144df7b5f5f5
  Stored in directory: /root/.cache/pip/wheels/5c/c0/b3/0c332de4fd71f3733ea6d61697464b7ae4b2b5ff0300e6ca7a
Successfully built keras.utils


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import librosa.display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import IPython
from tensorflow import keras
import keras.layers as L
#import tensorflow.keras.layers as L
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder,StandardScaler
import re
import itertools
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import os

In [6]:
Crema_Path='/content/drive/MyDrive/Datathon/Crema'
Ravdess_Path='/content/drive/MyDrive/Datathon/Ravdess/audio_speech_actors_01-24'
Savee_Path='/content/drive/MyDrive/Datathon/Savee'
Tess_Path='/content/drive/MyDrive/Datathon/Tess'

In [7]:
crema=[]
for wav in os.listdir(Crema_Path):
    emotion=wav.partition(".wav")[0].split('_')
    if emotion[2]=='SAD':
        crema.append(('sad',Crema_Path+'/'+wav))
    elif emotion[2]=='ANG':
        crema.append(('angry',Crema_Path+'/'+wav))
    elif emotion[2]=='DIS':
        crema.append(('disgust',Crema_Path+'/'+wav))
    elif emotion[2]=='FEA':
        crema.append(('fear',Crema_Path+'/'+wav))
    elif emotion[2]=='HAP':
        crema.append(('happy',Crema_Path+'/'+wav))
    elif emotion[2]=='NEU':
        crema.append(('neutral',Crema_Path+'/'+wav))
    else:
        crema.append(('unknown',Crema_Path+'/'+wav))
Crema_df=pd.DataFrame.from_dict(crema)
Crema_df.rename(columns={0:'Emotion',1:'File_Path'},inplace=True)
Crema_df.head()

,Emotion,File_Path
0,disgust,/content/drive/MyDrive/Datathon/Crema/1080_IWW...
1,disgust,/content/drive/MyDrive/Datathon/Crema/1079_TAI...
2,fear,/content/drive/MyDrive/Datathon/Crema/1081_IEO...
3,happy,/content/drive/MyDrive/Datathon/Crema/1079_WSI...
4,disgust,/content/drive/MyDrive/Datathon/Crema/1079_IEO...


In [8]:
ravdess=[]
for directory in os.listdir(Ravdess_Path):
    actors=os.listdir(os.path.join(Ravdess_Path,directory))
    for wav in actors:
        emotion=wav.partition('.wav')[0].split('-')
        emotion_number=int(emotion[2])
        ravdess.append((emotion_number,os.path.join(Ravdess_Path,directory,wav)))
Ravdess_df=pd.DataFrame.from_dict(ravdess)
Ravdess_df.rename(columns={0:'Emotion',1:'File_Path'},inplace=True)
Ravdess_df['Emotion'].replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'},inplace=True)
Ravdess_df.head()

,Emotion,File_Path
0,neutral,/content/drive/MyDrive/Datathon/Ravdess/audio_...
1,happy,/content/drive/MyDrive/Datathon/Ravdess/audio_...
2,happy,/content/drive/MyDrive/Datathon/Ravdess/audio_...
3,neutral,/content/drive/MyDrive/Datathon/Ravdess/audio_...
4,sad,/content/drive/MyDrive/Datathon/Ravdess/audio_...


In [9]:
savee=[]
for wav in os.listdir(Savee_Path):
    emo=wav.partition('.wav')[0].split('_')[1].replace(r'[0-9]','')
    emotion=re.split(r'[0-9]',emo)[0]
    if emotion=='a':
        savee.append(('angry',Savee_Path+'/'+wav))
    elif emotion=='d':
        savee.append(('disgust',Savee_Path+'/'+wav))
    elif emotion=='f':
        savee.append(('fear',Savee_Path+'/'+wav))
    elif emotion=='h':
        savee.append(('happy',Savee_Path+'/'+wav))
    elif emotion=='n':
        savee.append(('neutral',Savee_Path+'/'+wav))
    elif emotion=='sa':
        savee.append(('sad',Savee_Path+'/'+wav))
    elif emotion=='su':
        savee.append(('surprise',Savee_Path+'/'+wav))
Savee_df=pd.DataFrame.from_dict(savee)
Savee_df.rename(columns={0:'Emotion',1:'File_Path'},inplace=True)
Savee_df.head()

,Emotion,File_Path
0,angry,/content/drive/MyDrive/Datathon/Savee/DC_a08.wav
1,angry,/content/drive/MyDrive/Datathon/Savee/DC_a03.wav
2,angry,/content/drive/MyDrive/Datathon/Savee/DC_a10.wav
3,angry,/content/drive/MyDrive/Datathon/Savee/DC_a02.wav
4,angry,/content/drive/MyDrive/Datathon/Savee/DC_a01.wav


In [10]:
tess=[]
for directory in os.listdir(Tess_Path):
    for wav in os.listdir(os.path.join(Tess_Path,directory)):
        emotion=wav.partition('.wav')[0].split('_')
        if emotion[2]=='ps':
            tess.append(('surprise',os.path.join(Tess_Path,directory,wav)))
        else:
            tess.append((emotion[2],os.path.join(Tess_Path,directory,wav)))
Tess_df=pd.DataFrame.from_dict(tess)
Tess_df.rename(columns={0:'Emotion',1:'File_Path'},inplace=True)
Tess_df.head()

,Emotion,File_Path
0,fear,/content/drive/MyDrive/Datathon/Tess/OAF_Fear/...
1,fear,/content/drive/MyDrive/Datathon/Tess/OAF_Fear/...
2,fear,/content/drive/MyDrive/Datathon/Tess/OAF_Fear/...
3,fear,/content/drive/MyDrive/Datathon/Tess/OAF_Fear/...
4,fear,/content/drive/MyDrive/Datathon/Tess/OAF_Fear/...


In [11]:
main_df=pd.concat([Crema_df,Ravdess_df,Savee_df,Tess_df],axis=0)
main_df.shape

(12162, 2)

In [12]:
def add_noise(data,random=False,rate=0.035,threshold=0.075):
    if random:
        rate=np.random.random()*threshold
    noise=rate*np.random.uniform()*np.amax(data)
    augmented_data=data+noise*np.random.normal(size=data.shape[0])
    return augmented_data

def shifting(data,rate=1000):
    augmented_data=int(np.random.uniform(low=-5,high=5)*rate)
    augmented_data=np.roll(data,augmented_data)
    return augmented_data

def pitching(data,sr,pitch_factor=0.7,random=False):
    if random:
        pitch_factor=np.random.random() * pitch_factor
    return librosa.effects.pitch_shift(data,sr=sr,n_steps=pitch_factor)

def streching(data,rate=0.8):
    return librosa.effects.time_stretch(data,rate)

In [13]:
def zcr(data,frame_length,hop_length):
    zcr=librosa.feature.zero_crossing_rate(data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr,frame_length=2048,hop_length=512):
    result=np.array([])

    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

def get_features(path,duration=2.5, offset=0.6):
    data,sr=librosa.load(path,duration=duration,offset=offset)
    aud=extract_features(data,sr)
    audio=np.array(aud)

    noised_audio=add_noise(data,random=True)
    aud2=extract_features(noised_audio,sr)
    audio=np.vstack((audio,aud2))

    pitched_audio=pitching(data,sr,random=True)
    aud3=extract_features(pitched_audio,sr)
    audio=np.vstack((audio,aud3))

    pitched_audio1=pitching(data,sr,random=True)
    pitched_noised_audio=add_noise(pitched_audio1,random=True)
    aud4=extract_features(pitched_noised_audio,sr)
    audio=np.vstack((audio,aud4))

    return audio

In [ ]:
X,Y=[],[]
for path,emotion,index in zip(main_df.File_Path,main_df.Emotion,range(main_df.File_Path.shape[0])):
    features=get_features(path)
    if index%500==0:
        print(f'{index} audio has been processed')
    for i in features:
        X.append(i)
        Y.append(emotion)
print('Done')

In [52]:
import os

In [15]:
processed_data_path='/content/drive/MyDrive/Datathon/processed_data.csv'

In [60]:
extract=pd.DataFrame(X)
extract['Emotion']=Y
extract.to_csv(processed_data_path,index=False)
extract.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352,2353,2354,2355,2356,2357,2358,2359,2360,2361,2362,2363,2364,2365,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375,Emotion
0,0.010742,0.017090,0.022461,0.022461,0.028320,0.030273,0.029785,0.030273,0.030762,0.030762,0.034668,0.034668,0.031250,0.030273,0.029785,0.030273,0.030762,0.035156,0.036621,0.049805,0.049805,0.046387,0.048828,0.037109,0.038574,0.039551,0.037109,0.036621,0.041992,0.040039,0.041992,0.042969,0.040039,0.047363,0.051270,0.060059,0.065430,0.062012,0.056641,0.044434,0.037109,0.035645,0.041992,0.048340,0.051270,0.055176,0.049316,0.048828,0.047852,0.041992,...,-4.641767,6.766635,-16.658510,4.222315,0.467077,-5.531452,-7.513547,14.285885,-1.317500,-253.577835,154.938217,-67.304390,20.167511,-28.917892,3.347594,-19.577814,17.541389,-4.974300,21.065321,13.870062,-3.817768,9.145650,-14.079048,4.055459,3.156253,-0.354917,-10.952438,10.236942,-1.467523,-262.385498,155.405243,-68.552803,10.832536,-27.193569,7.007451,-17.170433,15.875989,-10.772264,17.382553,12.647110,-3.696645,10.080883,-12.304649,3.347931,3.738070,-1.177495,-18.989233,9.280723,4.093888,disgust
1,0.225098,0.328613,0.445312,0.453125,0.441406,0.448242,0.427246,0.411621,0.427734,0.420898,0.434082,0.435059,0.407227,0.404785,0.410645,0.398926,0.417480,0.411621,0.399414,0.405273,0.402344,0.409668,0.407227,0.412109,0.408203,0.417969,0.419922,0.415039,0.416992,0.409668,0.398438,0.389648,0.371094,0.324707,0.269043,0.214355,0.152344,0.113770,0.098633,0.069824,0.076172,0.097168,0.120117,0.143066,0.160645,0.187012,0.181152,0.205078,0.202148,0.187012,...,4.517926,-6.582789,-6.225937,-0.288588,-2.151200,-5.138868,-3.433813,6.558389,3.227670,-166.249081,56.867759,-5.426330,-13.661344,-9.220349,-10.842096,-2.964317,3.015533,7.564208,13.821954,10.424795,4.667145,-2.464214,-5.234092,2.008201,-0.218261,-2.548776,-4.209174,3.097828,-0.314320,-172.983697,53.715588,-8.426395,-20.555284,-8.118996,-5.830995,1.133331,0.283263,3.191155,10.341457,11.094946,7.532453,0.071173,-4.634231,3.155746,0.300362,-4.522783,-6.480361,1.187290,7.141549,disgust
2,0.008789,0.021973,0.031738,0.036621,0.035156,0.028320,0.028320,0.025879,0.032715,0.036133,0.033203,0.032715,0.029297,0.028809,0.032715,0.034668,0.034668,0.033691,0.030273,0.038086,0.037598,0.036621,0.036133,0.029785,0.037109,0.037598,0.041016,0.036133,0.033203,0.035645,0.036621,0.044434,0.048828,0.052734,0.057617,0.063965,0.066895,0.062500,0.062012,0.052734,0.042969,0.046387,0.046387,0.054688,0.057129,0.055176,0.052734,0.052246,0.051758,0.048828,...,-3.492770,11.937731,-13.476744,3.564250,-4.282421,-4.739622,-6.817904,15.696721,0.247017,-287.635498,154.987869,-70.297363,24.963177,-30.172260,2.667348,-22.801723,21.233131,-1.650508,21.788334,12.475316,-4.111681,10.865353,-10.527165,3.889011,-1.364988,-2.612342,-10.354553,17.315580,1.580661,-317.083344,143.483841,-62.595406,9.346053,-23.551121,-0.506037,-16.567617,17.285095,-0.738371,17.838768,12.986870,-2.831919,9.060424,-7.760564,2.087275,4.887050,-1.450094,-12.941345,11.169678,1.038688,disgust
3,0.058594,0.103027,0.160645,0.194824,0.192383,0.201660,0.191895,0.189453,0.209473,0.198730,0.199707,0.172363,0.155273,0.165039,0.152344,0.167969,0.178711,0.168457,0.174316,0.194824,0.182129,0.168457,0.155762,0.132812,0.161133,0.155762,0.169922,0.148926,0.139648,0.161133,0.165039,0.168945,0.156250,0.130371,0.100098,0.092773,0.076172,0.068359,0.065918,0.059570,0.052734,0.056641,0.064941,0.068848,0.074707,0.076660,0.071289,0.078125,0.074219,0.075195,...,-0.052932,1.463526,-4.298164,-3.751016,1.944035,-10.483180,-2.325641,12.227849,3.801007,-248.300087,100.152069,-27.227023,-9.875241,-5.404083,-16.722739,-3.949470,6.311619,10.467157,18.215042,9.929221,0.792435,0.757859,-3.438715,-2.798667,5.155040,-8.533221,-5.319384,11.690373,2.89

In [16]:
df=pd.read_csv(processed_data_path)
df.shape

(48648, 2377)

In [17]:
df=df.fillna(0)
print(df.isna().any())
df.shape

0          False
1          False
2          False
3          False
4          False
           ...  
2372       False
2373       False
2374       False
2375       False
Emotion    False
Length: 2377, dtype: bool


(48648, 2377)

In [18]:
X=df.drop(labels='Emotion',axis=1)
Y=df['Emotion']

In [19]:
lb=LabelEncoder()
Y=to_categorical(lb.fit_transform(Y))
print(lb.classes_)
Y

['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [40]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,random_state=42,test_size=0.2,shuffle=True)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((38918, 2376), (9730, 2376), (38918, 7), (9730, 7))

In [21]:
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,random_state=42,test_size=0.1,shuffle=True)
X_train.shape, X_test.shape, X_val.shape, y_train.shape,y_test.shape,y_val.shape

((35026, 2376), (9730, 2376), (3892, 2376), (35026, 7), (9730, 7), (3892, 7))

In [22]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
X_val=scaler.transform(X_val)
X_train.shape,X_test.shape,X_val.shape,y_train.shape,y_test.shape,y_val.shape

((35026, 2376), (9730, 2376), (3892, 2376), (35026, 7), (9730, 7), (3892, 7))

In [23]:
X_train=np.expand_dims(X_train,axis=2)
X_val=np.expand_dims(X_val,axis=2)
X_test=np.expand_dims(X_test,axis=2)
X_train.shape, X_test.shape, X_val.shape

((35026, 2376, 1), (9730, 2376, 1), (3892, 2376, 1))

In [24]:
early_stop=EarlyStopping(monitor='val_acc',mode='auto',patience=5,restore_best_weights=True)
lr_reduction=ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.5,min_lr=0.00001)

In [25]:
EPOCH=50
BATCH_SIZE=64

In [26]:
model=tf.keras.Sequential([
    L.Conv1D(512,kernel_size=5, strides=1,padding='same', activation='relu',input_shape=(X_train.shape[1],1)),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),
    L.Conv1D(512,kernel_size=5,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),
    L.Conv1D(256,kernel_size=5,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),
    L.Conv1D(256,kernel_size=3,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=5,strides=2,padding='same'),
    L.Conv1D(128,kernel_size=3,strides=1,padding='same',activation='relu'),
    L.BatchNormalization(),
    L.MaxPool1D(pool_size=3,strides=2,padding='same'),
    L.Flatten(),
    L.Dense(512,activation='relu'),
    L.BatchNormalization(),
    L.Dense(7,activation='softmax')
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2376, 512)         3072      
                                                                 
 batch_normalization (Batch  (None, 2376, 512)         2048      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 1188, 512)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 1188, 512)         1311232   
                                                                 
 batch_normalization_1 (Bat  (None, 1188, 512)         2048      
 chNormalization)                                                
                                                        

In [27]:
history=model.fit(X_train, y_train, epochs=EPOCH, validation_data=(X_val,y_val), batch_size=BATCH_SIZE,callbacks=[early_stop,lr_reduction])

Epoch 1/50
548/548 [==============================] - ETA: 0s - loss: 1.4066 - accuracy: 0.4685

548/548 [==============================] - 196s 330ms/step - loss: 1.4066 - accuracy: 0.4685 - val_loss: 1.3720 - val_accuracy: 0.4448 - lr: 0.0010
Epoch 2/50
548/548 [==============================] - ETA: 0s - loss: 1.0885 - accuracy: 0.5764

548/548 [==============================] - 178s 326ms/step - loss: 1.0885 - accuracy: 0.5764 - val_loss: 1.3132 - val_accuracy: 0.5108 - lr: 0.0010
Epoch 3/50
548/548 [==============================] - ETA: 0s - loss: 0.9457 - accuracy: 0.6376

548/548 [==============================] - 179s 326ms/step - loss: 0.9457 - accuracy: 0.6376 - val_loss: 1.4202 - val_accuracy: 0.5087 - lr: 0.0010
Epoch 4/50
548/548 [==============================] - ETA: 0s - loss: 0.8177 - accuracy: 0.6901

548/548 [==============================] - 183s 335ms/step - loss: 0.8177 - accuracy: 0.6901 - val_loss: 0.9481 - val_accuracy: 0.6524 - lr: 0.0010
Epoch 5/50
548/548 [==============================] - ETA: 0s - loss: 0.6778 - accuracy: 0.7498

...

548/548 [==============================] - 184s 335ms/step - loss: 0.0104 - accuracy: 0.9969 - val_loss: 0.2392 - val_accuracy: 0.9445 - lr: 0.0010
Epoch 47/50
548/548 [==============================] - ETA: 0s - loss: 0.0150 - accuracy: 0.9953

548/548 [==============================] - 178s 325ms/step - loss: 0.0150 - accuracy: 0.9953 - val_loss: 0.2405 - val_accuracy: 0.9427 - lr: 0.0010
Epoch 48/50
548/548 [==============================] - ETA: 0s - loss: 0.0239 - accuracy: 0.9925

548/548 [==============================] - 178s 326ms/step - loss: 0.0239 - accuracy: 0.9925 - val_loss: 0.2868 - val_accuracy: 0.9373 - lr: 0.0010
Epoch 49/50
548/548 [==============================] - ETA: 0s - loss: 0.0220 - accuracy: 0.9930

548/548 [==============================] - 178s 325ms/step - loss: 0.0220 - accuracy: 0.9930 - val_loss: 0.2847 - val_accuracy: 0.9337 - lr: 0.0010
Epoch 50/50
548/548 [==============================] - ETA: 0s - loss: 0.0117 - accuracy: 0.9966

548/548 [==============================] - 178s 325ms/step - loss: 0.0117 - accuracy: 0.9966 - val_loss: 0.2307 - val_accuracy: 0.9486 - lr: 0.0010


In [32]:
fig=px.line(history.history,y=['accuracy','val_accuracy'],
           labels={'index':'epoch','value':'accuracy'},
           title=f'According to the epoch accuracy and validation accuracy chart for the model')
fig.show()

In [29]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_pred

305/305 [==============================] - 16s 48ms/step


array([3, 3, 0, ..., 4, 4, 3])

In [41]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
loss,accuracy=model.evaluate(X_test,y_test,verbose=0)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

Test Loss: 0.22234848141670227
Test Accuracy: 0.9519013166427612


In [31]:
model_path = "/content/drive/MyDrive/Datathon/res_model.h5"
model.save(model_path)